This example is an experiment to validate that indeed we get better MSE using the real noise rather then the estimated one.
It's a sort of a sanity check for the algorithm.

In [8]:
from src.motion_models.trajectory_generation.route_generation import *
from src.visual.visual_tools import *

import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## CV Model ##

In [9]:
from src.filters import InitializationData, CVKalmanFilter

REAL_NOISE = np.array([0.2, 0.2])
REAL_PROCESS_NOISE_STD = np.array([0.5,0.5])
dt=0.04
white_accel_density = 1e1
T = 200
train_seed = 44
test_seed = 45

initial_velocity = np.array([14,5])
initial_position = np.array([1,5])

# Generate trajectories with the real noise
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=initial_trajectory_state, number_of_trajectories=40, measurement_noise_std=REAL_NOISE, vel_change_std=REAL_PROCESS_NOISE_STD, seed=train_seed)

# Initialize the noise std with large unkonwn value
INITIAL_NOISE = np.array([10, 10])
INITIAL_PROCESS_NOISE_STD = np.array([10,10])

# Initialzing the KF
initial_matrices_data = InitializationData(observation_noise_std=INITIAL_NOISE, process_noise_std=INITIAL_PROCESS_NOISE_STD)
estimated_kf = CVKalmanFilter(2, 4, 2, dt, initial_matrices_data)
estimated_kf.initialize()

# R fitting with fixed old_em method
print("Estimating R")
print(f"True R should be: {np.diag(REAL_NOISE**2)}")
print(f"Initial R: {estimated_kf.kf.R}")

for i in range(40):
    traj = trajectories[i][0]
    for j in range(3):
        # Estimate R
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=False, estimate_R=True)
    print(f"R : {estimated_kf.kf.R}")

print("Estimating Q")
for i in range(40):
    traj = trajectories[i][0]
    
    for j in range(20):
        # Estimate Q
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=True, estimate_R=False)
    print(f"Q : {estimated_kf.kf.Q}")


estimated_Q = estimated_kf.kf.Q
estimated_R = estimated_kf.kf.R
#plot_trajectories_with_predictions_shifted(noisy=test_traj, clean=clean_test_traj, predictions=[(preds, 1,)])


Estimating R
True R should be: [[0.04 0.  ]
 [0.   0.04]]
Initial R: [[100   0]
 [  0 100]]
R : [[ 7.47357474e+01 -9.40217678e-04]
 [-9.40217678e-04  7.47356529e+01]]
R : [[ 5.59628772e+01 -3.69101153e-05]
 [-3.69101153e-05  5.59619630e+01]]
R : [[4.19968768e+01 4.09113382e-03]
 [4.09113382e-03 4.19953562e+01]]
R : [[3.15845233e+01 3.21096277e-03]
 [3.21096277e-03 3.15830527e+01]]
R : [[2.38102774e+01 3.54518610e-03]
 [3.54518610e-03 2.38090781e+01]]
R : [[1.79955174e+01 3.51608660e-03]
 [3.51608660e-03 1.79955055e+01]]
R : [[1.36382898e+01 2.31129565e-03]
 [2.31129565e-03 1.36381725e+01]]
R : [[1.03701595e+01 1.26384001e-03]
 [1.26384001e-03 1.03682238e+01]]
R : [[7.90789926e+00 1.38795738e-03]
 [1.38795738e-03 7.90628665e+00]]
R : [[6.05084390e+00 7.52420689e-04]
 [7.52420689e-04 6.05124561e+00]]
R : [[ 4.64690764e+00 -2.73825254e-04]
 [-2.73825254e-04  4.64758504e+00]]
R : [[3.58305528e+00 2.78646303e-04]
 [2.78646303e-04 3.58277016e+00]]
R : [[ 2.77543391e+00 -1.60085014e-04]
 [-1.

In [23]:
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
test_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=initial_trajectory_state, number_of_trajectories=40, measurement_noise_std=REAL_NOISE, vel_change_std=REAL_PROCESS_NOISE_STD, seed=test_seed)

real_kf_mses = []
estimated_kf_mses = []

real_initial_matrices_data = InitializationData(observation_noise_std=REAL_NOISE, process_noise_std=REAL_PROCESS_NOISE_STD)
real_kf = CVKalmanFilter(2, 4, 2, dt, real_initial_matrices_data)
real_kf.initialize()

for noisy, clean, _ in test_trajectories:
    print("========")
    est_mse = estimated_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    real_mse = real_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    print(f"Estimated MSE: {est_mse}")
    print(f"Real MSE:      {real_mse}")
    real_kf_mses.append(real_mse)
    estimated_kf_mses.append(est_mse)
    
    real_kf.initialize()
    estimated_kf.initialize()
    estimated_kf.kf.R = estimated_R
    estimated_kf.kf.Q = estimated_Q

print("Real MSE mean:")
print(np.mean(np.array(real_kf_mses)))

print("Estimated MSE mean:")
print(np.mean(np.array(estimated_kf_mses)))


Estimated MSE: 0.12558357553976063
Real MSE:      0.1251503811137771
Estimated MSE: 0.13667969488913176
Real MSE:      0.136052396069208
Estimated MSE: 0.15332637959936302
Real MSE:      0.15161066100502296
Estimated MSE: 0.12436143817933852
Real MSE:      0.12479177562588842
Estimated MSE: 0.12266505928871721
Real MSE:      0.12280569182748836
Estimated MSE: 0.12648809709798142
Real MSE:      0.1255053469472581
Estimated MSE: 0.1256451239634635
Real MSE:      0.125991602301283
Estimated MSE: 0.1356221544397315
Real MSE:      0.13592236395747648
Estimated MSE: 0.1586683929890761
Real MSE:      0.1574316821925805
Estimated MSE: 0.12788847408672846
Real MSE:      0.12776242109379748
Estimated MSE: 0.14358589497570323
Real MSE:      0.14324194582884833
Estimated MSE: 0.11942910069816802
Real MSE:      0.11881705520704668
Estimated MSE: 0.12008904822121656
Real MSE:      0.1197799019236963
Estimated MSE: 0.14062979820541638
Real MSE:      0.140580585118205
Estimated MSE: 0.1349680078228139

## CA Model ##

In [27]:
from src.filters import InitializationData, CAKalmanFilter

dt = 0.04
REAL_NOISE = np.array([0.4,0.4])
white_accel_density = 5
real_accel_noise_std = np.sqrt(dt * white_accel_density)
initial_acceleration = np.array([1, 1])
initial_velocity = np.array([3,5])
initial_position = np.array([0,0])
T = 200

initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity, acceleration=initial_acceleration)
trajectories = generate_ca_trajectory(T=T, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=REAL_NOISE, number_of_trajectories=40, seed=train_seed, accel_noise_std=real_accel_noise_std)

# Initialize the noise std with large unkonwn value
INITIAL_NOISE = np.array([10, 10])
INITIAL_WHITE_ACCEL_DENSITY = 10

# Initialzing the KF
initial_matrices_data = InitializationData(observation_noise_std=INITIAL_NOISE, white_accel_density=INITIAL_WHITE_ACCEL_DENSITY)
estimated_kf = CAKalmanFilter(2, 6, 2, dt, initial_matrices_data)
estimated_kf.initialize()

# R fitting with fixed old_em method
print("Estimating R")
print(f"True R should be: {np.diag(REAL_NOISE**2)}")
print(f"Initial R: {estimated_kf.kf.R}")

for i in range(40):
    traj = trajectories[i][0]
    for j in range(3):
        # Estimate R
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=False, estimate_R=True)
    print(f"R : {estimated_kf.kf.R}")

print("Estimating Q")
for i in range(40):
    traj = trajectories[i][0]
    
    for j in range(20):
        # Estimate Q
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=True, estimate_R=False)
    print(f"Q : {estimated_kf.kf.Q}")


estimated_Q = estimated_kf.kf.Q
estimated_R = estimated_kf.kf.R
#plot_trajectories_with_predictions_shifted(noisy=test_traj, clean=clean_test_traj, predictions=[(preds, 1,)])

print(estimated_R)
print(estimated_Q)

Estimating R
True R should be: [[0.16 0.  ]
 [0.   0.16]]
Initial R: [[100   0]
 [  0 100]]
R : [[7.35939121e+01 4.20830795e-02]
 [4.20830795e-02 7.34667591e+01]]
R : [[54.30070905  0.13505814]
 [ 0.13505814 54.05512535]]
R : [[40.07663064  0.21608769]
 [ 0.21608769 39.84281869]]
R : [[29.51528928  0.13463471]
 [ 0.13463471 29.36948059]]
R : [[21.74849215  0.10942063]
 [ 0.10942063 21.65172897]]
R : [[16.03963633  0.07751382]
 [ 0.07751382 16.02537146]]
R : [[11.85540577  0.08329835]
 [ 0.08329835 11.88608908]]
R : [[8.79685519 0.06633386]
 [0.06633386 8.80053993]]
R : [[6.52472292 0.04243411]
 [0.04243411 6.54914937]]
R : [[4.8711919  0.03278063]
 [0.03278063 4.87420019]]
R : [[3.64817258 0.02732298]
 [0.02732298 3.64025405]]
R : [[2.73267298 0.02244304]
 [0.02244304 2.72807453]]
R : [[2.06644433 0.01540325]
 [0.01540325 2.04684076]]
R : [[1.57003099 0.01177625]
 [0.01177625 1.54935427]]
R : [[1.23645607 0.02319267]
 [0.02319267 1.19160344]]
R : [[0.95363961 0.01861856]
 [0.01861856 0

In [37]:
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
test_trajectories = generate_ca_trajectory(T=T, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=REAL_NOISE, number_of_trajectories=40, seed=test_seed, accel_noise_std=real_accel_noise_std)


real_kf_mses = []
estimated_kf_mses = []

real_initial_matrices_data = InitializationData(observation_noise_std=REAL_NOISE, white_accel_density=white_accel_density)
real_kf = CAKalmanFilter(2, 6, 2, dt, real_initial_matrices_data)
real_kf.initialize()

for noisy, clean, _ in test_trajectories:
    print("========")
    est_mse = estimated_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    real_mse = real_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    print(f"Estimated MSE: {est_mse}")
    print(f"Real MSE:      {real_mse}")
    real_kf_mses.append(real_mse)
    estimated_kf_mses.append(est_mse)
    
    real_kf.initialize()
    estimated_kf.initialize()
    estimated_kf.kf.R = estimated_R
    estimated_kf.kf.Q = estimated_Q

print("Real MSE mean:")
print(np.mean(np.array(real_kf_mses)))

print("Estimated MSE mean:")
print(np.mean(np.array(estimated_kf_mses)))


Estimated MSE: 0.4598552018834784
Real MSE:      0.460348985473397
Estimated MSE: 0.40059423543584555
Real MSE:      0.39704697283121704
Estimated MSE: 0.4473679781909269
Real MSE:      0.4431897725365669
Estimated MSE: 0.4903382054940516
Real MSE:      0.4837310955681714
Estimated MSE: 0.43551301958689415
Real MSE:      0.4307716322856344
Estimated MSE: 0.4926400270761316
Real MSE:      0.4907282669538702
Estimated MSE: 0.3920379321759738
Real MSE:      0.39052841286136075
Estimated MSE: 0.47546520057660224
Real MSE:      0.4707709191157758
Estimated MSE: 0.5755735062360283
Real MSE:      0.5735286939790486
Estimated MSE: 0.4153184187475518
Real MSE:      0.41496420260772393
Estimated MSE: 0.4599501831170152
Real MSE:      0.4559520041542628
Estimated MSE: 0.4141824600375713
Real MSE:      0.41882255160254217
Estimated MSE: 0.38729593891091313
Real MSE:      0.38451358468833946
Estimated MSE: 0.4845155766451121
Real MSE:      0.485347288960027
Estimated MSE: 0.573773553643923
Real MSE